<a href="https://colab.research.google.com/github/kokami236/osiro1/blob/main/11_17%E5%88%86%E5%89%B2%E7%82%B9%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install open3d tqdm

import open3d as o3d
import numpy as np
import os, random, shutil
from collections import defaultdict
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [6]:
# ==============================================================================
# SeedFormer / PCN 対応 データセット生成スクリプト
# ==============================================================================
# 機能:
# 1. 「完全点群(GT)」と「欠損点群(Hole)」の両方を読み込む
# 2. 同じ位置でスライスし、pair データ (partial / complete) を作成する
# 3. GTの重心を基準に正規化を行い、位置ズレを防ぐ
# 4. train/test リストを作成する
# 5. 途中再開(レジューム)機能付き
# ==============================================================================

import os
import sys
import glob
import numpy as np
import random
import itertools
import shutil
from tqdm import tqdm

# Google Colabで実行する場合、Open3Dが必要です
try:
    import open3d as o3d
except ImportError:
    print("Open3Dがインストールされていません。インストールします...")
    os.system("pip install open3d")
    import open3d as o3d

# ================================================
#  設定（ここを自分の環境に合わせて変更）
# ================================================

# 入力ファイル
GT_PLY   = "/content/drive/MyDrive/gakusyu/熊本正解データ.ply"  # 完全データ
HOLE_PLY = "/content/drive/MyDrive/gakusyu/kesson2.ply"     # 欠損データ

# 保存先ルートディレクトリ (★ここを変更しました)
# Google Driveの「マイドライブ」直下に、分かりやすい名前で保存します
# 好きな名前に書き換えてOKです
OUT_ROOT = "/content/drive/MyDrive/My_PCN_Dataset"

# カテゴリID（ShapeNet形式）
# 実際の保存先は .../My_PCN_Dataset/shapenet_pc/02691156/... のようになります
CATEGORY_ID = "02691156"

# パッチ作成設定
PATCH_SIZE = 0.25    # 1辺の長さ (単位: m)
STRIDE     = 0.1     # ずらし幅 (単位: m)
PATCH_N    = 2048    # 1パッチあたりの点数
MIN_POINTS = 50      # この点数未満はゴミとして捨てる

# 学習データの分割割合
TRAIN_SPLIT = 0.9    # 90% Train, 10% Test
RNG_SEED    = 42

# ================================================
#  初期化処理
# ================================================

if 'google.colab' in sys.modules:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ディレクトリ作成 (PCN形式: partial/complete)
BASE_DIR = f"{OUT_ROOT}/shapenet_pc/{CATEGORY_ID}"
PARTIAL_DIR  = os.path.join(BASE_DIR, "partial")
COMPLETE_DIR = os.path.join(BASE_DIR, "complete")
LIST_DIR     = f"{OUT_ROOT}/ShapeNet55_splits/Main_Split"

os.makedirs(PARTIAL_DIR, exist_ok=True)
os.makedirs(COMPLETE_DIR, exist_ok=True)
os.makedirs(LIST_DIR, exist_ok=True)

# ログファイル
LOG_FILE = f"{OUT_ROOT}/processed_log_{CATEGORY_ID}.txt"

# ================================================
#  1. 点群読み込み & KDTree構築
# ================================================
def load_pcd(path):
    print(f"Loading {path} ...")
    if not os.path.exists(path):
        print(f"❌ エラー: ファイルが見つかりません -> {path}")
        sys.exit(1)
    pcd = o3d.io.read_point_cloud(path)
    pts = np.asarray(pcd.points)
    print(f"  -> Points: {len(pts)}")
    return pcd, pts

# 完全データ(GT)と欠損データ(Hole)の両方を読み込み
gt_pcd, gt_pts = load_pcd(GT_PLY)
hole_pcd, hole_pts = load_pcd(HOLE_PLY)

# 探索範囲はGTを基準にする
global_min = gt_pts.min(axis=0)
global_max = gt_pts.max(axis=0)

print("Building KDTree for acceleration...")
# 両方のKDTreeを作成（高速切り出しのため）
tree_gt = o3d.geometry.KDTreeFlann(gt_pcd)
tree_hole = o3d.geometry.KDTreeFlann(hole_pcd)

# グリッド設定
margin = 0.05
x_steps = np.arange(global_min[0]-margin, global_max[0]+margin, STRIDE)
y_steps = np.arange(global_min[1]-margin, global_max[1]+margin, STRIDE)
z_steps = np.arange(global_min[2]-margin, global_max[2]+margin, STRIDE)

total_steps = len(x_steps) * len(y_steps) * len(z_steps)
print(f"Grid setup: Size={PATCH_SIZE}m, Stride={STRIDE}m")
print(f"  -> Total search grids: {total_steps}")

# ================================================
#  2. レジューム（再開）情報
# ================================================
processed_keys = set()
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "r") as f:
        for line in f:
            processed_keys.add(line.strip())
    print(f"🔄 Resuming... skipped {len(processed_keys)} grids.")

# 既存ファイルからID決定 (completeフォルダを基準にする)
existing_files = glob.glob(os.path.join(COMPLETE_DIR, "*.pcd"))
current_max_id = 0
if len(existing_files) > 0:
    try:
        ids = [int(os.path.basename(f).split('-')[-1].replace('.pcd','')) for f in existing_files]
        current_max_id = max(ids) + 1
    except:
        current_max_id = len(existing_files)

patch_id = current_max_id
print(f"Start generating from ID: {patch_id}")

# ================================================
#  3. パッチ生成メインループ
# ================================================
print("🚀 Start generating PAIRS (partial & complete)...")
grid_coords = itertools.product(x_steps, y_steps, z_steps)
new_count = 0
search_radius = (PATCH_SIZE * np.sqrt(3) / 2.0) * 1.01

def get_points_in_box(tree, all_pts, center, radius, min_b, max_b):
    # 半径探索
    [k, idx, _] = tree.search_radius_vector_3d(center, radius)
    if k < MIN_POINTS:
        return np.empty((0, 3))

    # ボックス判定
    cand = all_pts[idx]
    mask = np.all((cand >= min_b) & (cand < max_b), axis=1)
    return cand[mask]

def sample_points(pts, n_points):
    if len(pts) == 0: return pts
    if len(pts) >= n_points:
        return pts[np.random.choice(len(pts), n_points, replace=False)]
    else:
        return pts[np.random.choice(len(pts), n_points, replace=True)]

with open(LOG_FILE, "a") as log_f:
    for x, y, z in tqdm(grid_coords, total=total_steps, mininterval=1.0, desc="Slicing"):

        key = f"{x:.3f}_{y:.3f}_{z:.3f}"
        if key in processed_keys:
            continue

        center = np.array([x + PATCH_SIZE/2, y + PATCH_SIZE/2, z + PATCH_SIZE/2])
        min_bound = np.array([x, y, z])
        max_bound = min_bound + PATCH_SIZE

        # 1. Complete (GT) の切り出し
        gt_patch = get_points_in_box(tree_gt, gt_pts, center, search_radius, min_bound, max_bound)

        # GTが少なすぎる場合は学習データにならないのでスキップ
        if len(gt_patch) < MIN_POINTS:
            log_f.write(key + "\n")
            continue

        # 2. Partial (Hole) の切り出し
        hole_patch = get_points_in_box(tree_hole, hole_pts, center, search_radius, min_bound, max_bound)

        # Partial側が空っぽ(0点)でも、補完タスクとしては「全欠損」としてあり得るが、
        # 学習が難しくなるため、ある程度点があるものだけ採用するか、
        # ユーザーの要件次第。ここでは「Partialも少しは点がないと位置合わせできない」と判断しスキップ。
        if len(hole_patch) < MIN_POINTS:
            log_f.write(key + "\n")
            continue

        # 3. サンプリング
        gt_final = sample_points(gt_patch, PATCH_N)
        hole_final = sample_points(hole_patch, PATCH_N)

        # 4. ★正規化 (位置合わせ)
        # 重要: 「GTの中心」を使って両方をシフトする。
        # Partial自身の中心を使うと、PartialとCompleteの位置関係がズレてしまう。
        patch_center = np.mean(gt_final, axis=0)

        gt_final   = gt_final - patch_center
        hole_final = hole_final - patch_center

        # 5. 保存
        file_name = f"{CATEGORY_ID}-{patch_id:06d}.pcd"

        # Complete保存
        o3d.io.write_point_cloud(
            os.path.join(COMPLETE_DIR, file_name),
            o3d.geometry.PointCloud(o3d.utility.Vector3dVector(gt_final)),
            write_ascii=False
        )
        # Partial保存
        o3d.io.write_point_cloud(
            os.path.join(PARTIAL_DIR, file_name),
            o3d.geometry.PointCloud(o3d.utility.Vector3dVector(hole_final)),
            write_ascii=False
        )

        patch_id += 1
        new_count += 1
        log_f.write(key + "\n")

print(f"\n✨ 生成完了: {new_count} ペア (Total ID: {patch_id})")

# ================================================
#  4. リスト作成
# ================================================
print("\n📝 Generating split lists...")
all_files = sorted([os.path.basename(f) for f in glob.glob(os.path.join(COMPLETE_DIR, "*.pcd"))])

if len(all_files) > 0:
    random.shuffle(all_files)
    split_idx = int(len(all_files) * TRAIN_SPLIT)
    train_files = all_files[:split_idx]
    test_files  = all_files[split_idx:]

    with open(os.path.join(LIST_DIR, "train.txt"), "w") as f:
        f.write("\n".join(train_files))
    with open(os.path.join(LIST_DIR, "test.txt"), "w") as f:
        f.write("\n".join(test_files))

    print(f"✅ リスト作成完了: Train={len(train_files)}, Test={len(test_files)}")
else:
    print("⚠️ ファイルなし")

print("\n🎉 完了！")

Loading /content/drive/MyDrive/gakusyu/熊本正解データ.ply ...
  -> Points: 970651
Loading /content/drive/MyDrive/gakusyu/kesson2.ply ...
  -> Points: 468677
Building KDTree for acceleration...
Grid setup: Size=0.25m, Stride=0.1m
  -> Total search grids: 62424
Start generating from ID: 0
🚀 Start generating PAIRS (partial & complete)...


Slicing: 100%|██████████| 62424/62424 [01:30<00:00, 686.95it/s] 



✨ 生成完了: 6124 ペア (Total ID: 6124)

📝 Generating split lists...
✅ リスト作成完了: Train=5511, Test=613

🎉 完了！


In [6]:
import glob, os, shutil

# ============
# 1. パス設定
# ============
SRC_ROOT = '/content/drive/MyDrive/My_PCN_Dataset1/shapenet_pc/02691156'
DST_ROOT = '/content/My_PCN_Dataset/PCN/train'

# ==================
# 2. partial の一覧
# ==================
partial_files = sorted(glob.glob(os.path.join(SRC_ROOT, 'partial', '*.pcd')))
total = len(partial_files)

print(f"🔍 partial 点群の総数: {total}")

# ======================
# 3. PCN 形式へコピー
# ======================
processed = 0
skipped = 0

for idx, p_path in enumerate(partial_files, start=1):
    fname = os.path.basename(p_path)         # 例: 02691156-001437.pcd
    stem  = os.path.splitext(fname)[0]       # 例: 02691156-001437

    # 完成形（complete）を探す
    c_path = os.path.join(SRC_ROOT, 'complete', fname)

    # 進捗表示（スキップ前に出す）
    print(f"[{idx}/{total}] 処理中: {fname}")

    if not os.path.exists(c_path):
        print(f"   ➤ [SKIP] complete が存在しません → {fname}")
        skipped += 1
        continue

    # ---------- partial 側 ----------
    dst_partial_dir = os.path.join(DST_ROOT, 'partial', 'airplane', stem)
    os.makedirs(dst_partial_dir, exist_ok=True)
    shutil.copy(p_path, os.path.join(dst_partial_dir, '00.pcd'))

    # ---------- complete 側 ----------
    dst_complete_dir = os.path.join(DST_ROOT, 'complete', 'airplane')
    os.makedirs(dst_complete_dir, exist_ok=True)
    shutil.copy(c_path, os.path.join(dst_complete_dir, f"{stem}.pcd"))

    processed += 1
    print(f"   ✔ コピー完了: {stem}")

# ======================
# 4. 結果レポート
# ======================
print("\n=======================")
print("  ⭐ 変換処理 完了 ⭐")
print("=======================\n")
print(f"✔ 成功: {processed} 件")
print(f"⚠ スキップ: {skipped} 件")
print(f"📁 出力先: {DST_ROOT}")


ストリーミング出力は最後の 5000 行に切り捨てられました。
[3629/6124] 処理中: 02691156-003628.pcd
   ✔ コピー完了: 02691156-003628
[3630/6124] 処理中: 02691156-003629.pcd
   ✔ コピー完了: 02691156-003629
[3631/6124] 処理中: 02691156-003630.pcd
   ✔ コピー完了: 02691156-003630
[3632/6124] 処理中: 02691156-003631.pcd
   ✔ コピー完了: 02691156-003631
[3633/6124] 処理中: 02691156-003632.pcd
   ✔ コピー完了: 02691156-003632
[3634/6124] 処理中: 02691156-003633.pcd
   ✔ コピー完了: 02691156-003633
[3635/6124] 処理中: 02691156-003634.pcd
   ✔ コピー完了: 02691156-003634
[3636/6124] 処理中: 02691156-003635.pcd
   ✔ コピー完了: 02691156-003635
[3637/6124] 処理中: 02691156-003636.pcd
   ✔ コピー完了: 02691156-003636
[3638/6124] 処理中: 02691156-003637.pcd
   ✔ コピー完了: 02691156-003637
[3639/6124] 処理中: 02691156-003638.pcd
   ✔ コピー完了: 02691156-003638
[3640/6124] 処理中: 02691156-003639.pcd
   ✔ コピー完了: 02691156-003639
[3641/6124] 処理中: 02691156-003640.pcd
   ✔ コピー完了: 02691156-003640
[3642/6124] 処理中: 02691156-003641.pcd
   ✔ コピー完了: 02691156-003641
[3643/6124] 処理中: 02691156-003642.pcd
   ✔ コピー完了: 02691156-

In [7]:
!zip -r /content/My_PCN_Dataset_PCN.zip /content/My_PCN_Dataset/PCN


ストリーミング出力は最後の 5000 行に切り捨てられました。
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-005520.pcd (deflated 47%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-002563.pcd (deflated 9%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-001552.pcd (deflated 9%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-000857.pcd (deflated 37%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-004818.pcd (deflated 39%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-001569.pcd (deflated 9%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-004225.pcd (deflated 78%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-000905.pcd (deflated 9%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-002992.pcd (deflated 8%)
  adding: content/My_PCN_Dataset/PCN/train/complete/airplane/02691156-002595.pcd (deflated 9%)
  adding: cont

In [8]:
!cp -r /content/My_PCN_Dataset /content/drive/MyDrive/


In [4]:
import open3d as o3d
import numpy as np
import glob

# 生成されたファイルを1つ取得
pcd_files = glob.glob("/content/drive/MyDrive/SeedFormer_ShapeNet55_Data_25cm/shapenet_pc/02691156/*.pcd")
target_file = pcd_files[0]

print(f"Checking: {target_file}")
pcd = o3d.io.read_point_cloud(target_file)
pts = np.asarray(pcd.points)

print(f"座標の最大値: {pts.max(axis=0)}")
print(f"座標の最小値: {pts.min(axis=0)}")
print(f"中心座標: {np.mean(pts, axis=0)}")

# 期待される結果:
# 中心座標が [0. 0. 0.] (またはそれに非常に近い値) になっていれば成功です！

Checking: /content/drive/MyDrive/SeedFormer_ShapeNet55_Data_25cm/shapenet_pc/02691156/02691156-000000.pcd
座標の最大値: [0.04034611 0.04080145 0.05576121]
座標の最小値: [-0.10139039 -0.13602193 -0.05192501]
中心座標: [-9.41327016e-11  8.62883098e-11 -3.00701686e-10]


In [3]:
import os
import numpy as np
import open3d as o3d
from tqdm import tqdm
import random
import itertools
import glob

# ================================================
#   設定
# ================================================
GT_PLY    = "/content/drive/MyDrive/gakusyu/熊本正解データ.ply"
OUT_ROOT  = "/content/drive/MyDrive/SeedFormer_ShapeNet55_Data_25cm"
CATEGORY_ID = "02691156"

PATCH_SIZE = 0.25    # 25cm区画
STRIDE     = 0.1     # 5cmずつずらす
PATCH_N    = 2048    # 学習用の点数
MIN_POINTS = 50      # 最低限必要な点数

TRAIN_SPLIT = 0.9
RNG_SEED   = 42

# 途中経過を保存するログファイル
LOG_FILE = f"{OUT_ROOT}/processed_log.txt"

random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ================================================
#   ディレクトリ作成
# ================================================
PC_DIR = f"{OUT_ROOT}/shapenet_pc"
SPLIT_DIR = f"{OUT_ROOT}/splits"
os.makedirs(PC_DIR, exist_ok=True)
os.makedirs(SPLIT_DIR, exist_ok=True)

# ================================================
#   読み込み & 準備
# ================================================
print(f"Loading {GT_PLY}...")
if not os.path.exists(GT_PLY):
    print(f"エラー: ファイルが見つかりません -> {GT_PLY}")
    exit()

gt_pcd = o3d.io.read_point_cloud(GT_PLY)
gt_pts = np.asarray(gt_pcd.points)

if len(gt_pts) == 0:
    print("エラー: データが空です")
    exit()

# 座標正規化
mins = gt_pts.min(axis=0)
maxs = gt_pts.max(axis=0)
gt_norm = gt_pts - mins

# ★高速化の鍵: KDTreeの構築
# これにより「近くに点があるか」を一瞬で検索できます
print("Building KDTree for speedup...")
pcd_tree = o3d.geometry.KDTreeFlann(gt_pcd)

# 探索範囲の計算
search_max = maxs - mins
x_steps = np.arange(0, search_max[0], STRIDE)
y_steps = np.arange(0, search_max[1], STRIDE)
z_steps = np.arange(0, search_max[2], STRIDE)

print(f"区画サイズ: {PATCH_SIZE}m, ずらし幅: {STRIDE}m")
total_steps = len(x_steps) * len(y_steps) * len(z_steps)
print(f"探索グリッド数: {total_steps}")

# ================================================
#   レジューム（途中再開）の準備
# ================================================
processed_coords = set()

# ログファイルがあれば読み込む
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "r") as f:
        for line in f:
            processed_coords.add(line.strip())
    print(f"🔄 過去の履歴を読み込みました: {len(processed_coords)} 箇所済み")

# 現在のファイル数から開始IDを決定
existing_files = glob.glob(os.path.join(PC_DIR, "*.pcd"))
patch_id = len(existing_files)
print(f"Start ID: {patch_id}")

# ================================================
#   スライディングウィンドウ処理
# ================================================
print("Generating patches...")

grid_coords = itertools.product(x_steps, y_steps, z_steps)
new_generated_count = 0

# ログファイルを開く（追記モード）
with open(LOG_FILE, "a") as log_f:

    for x, y, z in tqdm(grid_coords, total=total_steps, mininterval=2.0, desc="Processing"):

        # 座標キー（文字列として管理）
        coord_key = f"{x:.3f}_{y:.3f}_{z:.3f}"

        # すでに処理済みならスキップ（高速化）
        if coord_key in processed_coords:
            continue

        # ボックスの中心と半径を計算
        min_bound = np.array([x, y, z])
        center = min_bound + PATCH_SIZE / 2.0
        # ボックスを包含する球の半径（少し余裕を持たせる）
        radius = (PATCH_SIZE * np.sqrt(3) / 2.0) * 1.05

        # ★高速化: KDTreeで「半径内にある点」のインデックスを取得
        # k: 見つかった点数, idx: インデックスリスト, _: 距離
        [k, idx, _] = pcd_tree.search_radius_vector_3d(center, radius)

        # 近くに点が少なければ、詳細判定するまでもなくスキップ
        if k < MIN_POINTS:
            # 処理済みとして記録（次回スキップするため）
            log_f.write(coord_key + "\n")
            continue

        # 候補点だけを取り出す
        candidate_pts = gt_norm[idx]

        # 厳密なボックス判定（矩形クリップ）
        # ここでの計算量は k個（少数）に対してだけなので非常に高速
        max_bound = min_bound + PATCH_SIZE
        mask = np.all((candidate_pts >= min_bound) & (candidate_pts < max_bound), axis=1)
        patch_pts = candidate_pts[mask]

        # 点数チェック
        if len(patch_pts) > MIN_POINTS:

            # サンプリング
            if len(patch_pts) >= PATCH_N:
                choice_idx = np.random.choice(len(patch_pts), PATCH_N, replace=False)
            else:
                choice_idx = np.random.choice(len(patch_pts), PATCH_N, replace=True)

            final_patch = patch_pts[choice_idx]

            # 中心化
            final_patch = final_patch - np.mean(final_patch, axis=0)

            # 保存
            file_name = f"{CATEGORY_ID}-{patch_id:06d}.pcd"
            save_path = os.path.join(PC_DIR, file_name)

            pcd_out = o3d.geometry.PointCloud()
            pcd_out.points = o3d.utility.Vector3dVector(final_patch)
            o3d.io.write_point_cloud(save_path, pcd_out, write_ascii=False)

            patch_id += 1
            new_generated_count += 1

        # 処理完了を記録（点があってもなくても、このグリッドは終わったとみなす）
        log_f.write(coord_key + "\n")

print(f"\n🎉 今回の実行で {new_generated_count} 個のパッチを作成しました！")
print(f"現在の総ファイル数: {patch_id}")

# ================================================
#   リスト作成（全ファイルを対象に再生成）
# ================================================
# フォルダ内の全PCDを取得しなおす（中断後の再開でも整合性をとるため）
all_files = sorted([os.path.basename(f) for f in glob.glob(os.path.join(PC_DIR, "*.pcd"))])

if len(all_files) > 0:
    random.shuffle(all_files)
    split_idx = int(len(all_files) * TRAIN_SPLIT)
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]

    with open(f"{SPLIT_DIR}/train.txt", "w") as f:
        for name in train_files:
            f.write(name + "\n")

    with open(f"{SPLIT_DIR}/test.txt", "w") as f:
        for name in test_files:
            f.write(name + "\n")
    print(f"リスト更新完了: Train={len(train_files)}, Test={len(test_files)}")
else:
    print("ファイルが見つかりません。")

Loading /content/drive/MyDrive/gakusyu/熊本正解データ.ply...
Building KDTree for speedup...
区画サイズ: 0.25m, ずらし幅: 0.1m
探索グリッド数: 57750
Start ID: 6920
Generating patches...


Processing: 100%|██████████| 57750/57750 [00:01<00:00, 55153.45it/s]



🎉 今回の実行で 0 個のパッチを作成しました！
現在の総ファイル数: 6920
リスト更新完了: Train=6228, Test=692


In [4]:
import os
import numpy as np
import open3d as o3d
from tqdm import tqdm
import random
import itertools # 追加: ループを効率化するため

# ================================================
#   設定（25cm区画に変更！）
# ================================================
GT_PLY    = "/content/drive/MyDrive/gakusyu/熊本正解データ.ply"
OUT_ROOT  = "/content/drive/MyDrive/SeedFormer_ShapeNet55_Data_25cm4"
CATEGORY_ID = "02691156"

PATCH_SIZE = 0.25    # 25cm区画
STRIDE     = 0.1     # 5cmずつずらす
PATCH_N    = 2048    # 学習用の点数
MIN_POINTS = 50      # 最低限必要な点数

TRAIN_SPLIT = 0.9
RNG_SEED   = 42

random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ================================================
#   ディレクトリ作成
# ================================================
PC_DIR = f"{OUT_ROOT}/shapenet_pc"
SPLIT_DIR = f"{OUT_ROOT}/splits"
os.makedirs(PC_DIR, exist_ok=True)
os.makedirs(SPLIT_DIR, exist_ok=True)

# ================================================
#   読み込み & 準備
# ================================================
print(f"Loading {GT_PLY}...")
if not os.path.exists(GT_PLY):
    print(f"エラー: ファイルが見つかりません -> {GT_PLY}")
    exit()

gt_pcd = o3d.io.read_point_cloud(GT_PLY)
gt_pts = np.asarray(gt_pcd.points)

if len(gt_pts) == 0:
    print("エラー: データが空です")
    exit()

# 座標正規化
mins = gt_pts.min(axis=0)
maxs = gt_pts.max(axis=0)
gt_norm = gt_pts - mins

# 探索範囲の計算
search_max = maxs - mins
x_steps = np.arange(0, search_max[0], STRIDE)
y_steps = np.arange(0, search_max[1], STRIDE)
z_steps = np.arange(0, search_max[2], STRIDE)

print(f"区画サイズ: {PATCH_SIZE}m, ずらし幅: {STRIDE}m")
total_steps = len(x_steps) * len(y_steps) * len(z_steps)
print(f"予測生成数（理論最大値）: {total_steps}")

# ================================================
#   スライディングウィンドウ処理
# ================================================
generated_files = []
patch_id = 0

print("Generating patches...")

# ★修正点: itertoolsを使って3重ループを1つに展開し、tqdmの表示頻度を抑制
# mininterval=2.0 にすることで、表示更新を2秒に1回に抑えます（ログが溢れるのを防ぐ）
grid_coords = itertools.product(x_steps, y_steps, z_steps)

for x, y, z in tqdm(grid_coords, total=total_steps, mininterval=2.0, desc="Processing"):

    # ボックスの定義
    min_bound = np.array([x, y, z])
    max_bound = min_bound + PATCH_SIZE

    # 範囲内の点を取得（Numpy高速化）
    # ブロードキャストを使って一括判定（少しメモリ食いますが高速です）
    mask = np.all((gt_norm >= min_bound) & (gt_norm < max_bound), axis=1)
    patch_pts = gt_norm[mask]

    # 点数チェック
    if len(patch_pts) > MIN_POINTS:

        # サンプリング（2048点に揃える）
        if len(patch_pts) >= PATCH_N:
            choice_idx = np.random.choice(len(patch_pts), PATCH_N, replace=False)
        else:
            choice_idx = np.random.choice(len(patch_pts), PATCH_N, replace=True)

        final_patch = patch_pts[choice_idx]

        # 中心化
        final_patch = final_patch - np.mean(final_patch, axis=0)

        # 保存
        file_name = f"{CATEGORY_ID}-{patch_id:06d}.pcd"
        save_path = os.path.join(PC_DIR, file_name)

        # PCD保存
        pcd_out = o3d.geometry.PointCloud()
        pcd_out.points = o3d.utility.Vector3dVector(final_patch)
        o3d.io.write_point_cloud(save_path, pcd_out, write_ascii=False)

        generated_files.append(file_name)
        patch_id += 1

print(f"\n🎉 合計 {len(generated_files)} 個のパッチを作成しました！")

# ================================================
#   リスト作成
# ================================================
if len(generated_files) > 0:
    random.shuffle(generated_files)
    split_idx = int(len(generated_files) * TRAIN_SPLIT)
    train_files = generated_files[:split_idx]
    test_files = generated_files[split_idx:]

    with open(f"{SPLIT_DIR}/train.txt", "w") as f:
        for name in train_files:
            f.write(name + "\n")

    with open(f"{SPLIT_DIR}/test.txt", "w") as f:
        for name in test_files:
            f.write(name + "\n")
    print(f"リスト保存完了: Train={len(train_files)}, Test={len(test_files)}")
    print(f"保存先: {OUT_ROOT}")
else:
    print("パッチが生成されませんでした。STRIDEを小さくするか、MIN_POINTSを下げてください。")

Loading /content/drive/MyDrive/gakusyu/熊本正解データ.ply...
区画サイズ: 0.25m, ずらし幅: 0.1m
予測生成数（理論最大値）: 57750
Generating patches...


Processing: 100%|██████████| 57750/57750 [29:58<00:00, 32.11it/s]


🎉 合計 6920 個のパッチを作成しました！
リスト保存完了: Train=6228, Test=692
保存先: /content/drive/MyDrive/SeedFormer_ShapeNet55_Data_25cm4


環境構築ここからランタイムの切り替え

